In [ ]:
from __future__ import print_function
import os
os.environ["CUDA_VISIBLE_DEVICES"] = ""

import sys
import math
import json
import pickle
import numpy as np

In [ ]:
import json
sentence_list_fname = './assignment_training_data_word_segment.json'
sentence_list = json.load(open(sentence_list_fname , 'r',encoding='utf-8'))

In [ ]:
from gensim.models import Word2Vec
import multiprocessing
word = []
for index in range(3000):
    a = sentence_list[index]['indexes']
    b = [str(i) for i in a]
    word.append(b)
#word = ' '.join(word)
print(word)

L = 10
print('training...')
model = Word2Vec(word, size=L, window=5, min_count=5, workers=multiprocessing.cpu_count(),
                 iter=10)
# model.save('Word2VecModel.m')
#model.wv.save_word2vec_format('Word2VecModel.vector', binary=False)
print('outputing...')

print(model)
print(model['0'])


In [ ]:
sentence = sentence_list[0]
print (sentence)



In [ ]:
max_length = 0
for index in range(3000):
    temp_len = len(sentence_list[index]['words']) 
    if temp_len > max_length:
        max_length = temp_len
print(max_length)

all_result = 0
for index in range(3000):
    temp_len = len(sentence_list[index]['results']) 
    all_result += temp_len
print("1-result:",all_result)

all_result_0 = 0
for index in range(3000):
    temp_len = len(sentence_list[index]['times'])*len(sentence_list[index]['values'])*len(sentence_list[index]['attributes'])
    all_result_0 += temp_len
print("0-result:",all_result_0 - all_result)

print("result:", all_result_0)

import random
true_num = 0
not_true_num = 0
data = []
data_y = []
print(len(data))
total_index = 0
for index in range(3000):
    for time in sentence_list[index]['times']:
        for attribute in sentence_list[index]['attributes']:
            for value in sentence_list[index]['values']:
                condition = [time,attribute,value]
                if condition in sentence_list[index]['results']:
                    true_flag = 1
                    true_num += 1
                    data_y.append([1,0])
                else:
                    true_flag = 0
                    r_del = random.randint(0, 18)
                    if r_del != 0:
                        continue
                    not_true_num += 1
                    data_y.append([0,1])
                word_temp = []
                sen_vec = []
                word_index = 0
                for word in sentence_list[index]['indexes']:
                    if (true_flag == 1) and (word_index in condition):
                        add_item = 10
                    else:
                        add_item = 0
                    if str(word) in model:
                        word_vec = list(model[str(word)])
                        word_vec.append(add_item)
                        word_vec.append(add_item)
                        word_vec.append(add_item)
                        word_vec.append(add_item)
                    else:
                        word_vec = [0]*14
                    sen_vec += word_vec 
                    word_index += 1
                while len(sen_vec) < 2156:
                    sen_vec += [0]*14
                data.append(sen_vec)
                

print(len(data))
print(len(data_y))
print(true_num)
print(not_true_num)

In [ ]:
raw_x = np.array(data)
raw_y = np.array(data_y)
print(raw_x.shape)
print(raw_y.shape)

In [ ]:
#划分出训练集和验证集
from sklearn.model_selection import train_test_split
x_train, x_validation, y_train, y_validation = train_test_split(raw_x, raw_y, test_size=0.30, random_state=6)
print(x_train.shape)
print(x_validation.shape)
print(y_train.shape)
print(y_validation.shape)

In [ ]:
import  tensorflow as tf
from tensorflow.contrib import  rnn

In [ ]:
import random
def get_batch(x,y, batch_size):
    total_len = len(x)
    begin_index = random.randint(0, total_len - batch_size)
    end_index = begin_index + batch_size
    select_x = x[begin_index:end_index,:]
    select_y = y[begin_index:end_index,:]
    return (select_x, select_y)

In [ ]:

trainimgs, trainlabels, testimgs, testlabels = x_train, y_train, x_validation, y_validation
ntrain, ntest, dim, nclasses =trainimgs.shape[0], testimgs.shape[0],trainimgs.shape[1],trainlabels.shape[1]
print(ntrain, ntest, dim, nclasses)
print ("MNIST loaded")
print(trainimgs.shape)
print(trainlabels.shape)


x = get_batch(trainimgs,trainlabels, 1280)
print(x[0].shape)


In [ ]:
train_rate=0.01
train_step=100
batch_size=1280
display_step=1

frame_size=14
sequence_length=154
hidden_num=200
n_classes=2

#定义输入,输出
x=tf.placeholder(dtype=tf.float32,shape=[None,sequence_length*frame_size],name="inputx")
y=tf.placeholder(dtype=tf.float32,shape=[None,n_classes],name="expected_y")
#定义权值
weights=tf.Variable(tf.truncated_normal(shape=[hidden_num,n_classes]))
bias=tf.Variable(tf.zeros(shape=[n_classes]))

def RNN(x,weights,bias):
    x=tf.reshape(x,shape=[-1,sequence_length,frame_size])
    rnn_cell=tf.nn.rnn_cell.BasicRNNCell(hidden_num)
    init_state=tf.zeros(shape=[batch_size,rnn_cell.state_size])
    output,states=tf.nn.dynamic_rnn(rnn_cell,x,dtype=tf.float32)
    return tf.nn.softmax(tf.matmul(output[:,-1,:],weights)+bias,1)
predy=RNN(x,weights,bias)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=predy,labels=y))
train=tf.train.AdamOptimizer(train_rate).minimize(cost)

correct_pred=tf.equal(tf.argmax(predy,1),tf.argmax(y,1))
accuracy=tf.reduce_mean(tf.to_float(correct_pred))

sess=tf.Session()
sess.run(tf.initialize_all_variables())
step=1
testx,testy=get_batch(testimgs,testlabels, batch_size)
while step<train_step:
    batch_x,batch_y=get_batch(trainimgs,trainlabels, batch_size)
#    batch_x=tf.reshape(batch_x,shape=[batch_size,sequence_length,frame_size])
    _loss,__=sess.run([cost,train],feed_dict={x:batch_x,y:batch_y})
    if step % display_step ==0:

        acc,loss=sess.run([accuracy,cost],feed_dict={x:testx,y:testy})
        print(step,acc,loss)

    step+=1

In [ ]:
print(x_train[0])

In [ ]:

acc,loss=sess.run([accuracy,cost],feed_dict={x:x_validation,y:y_validation})
print(acc,loss)